In [2]:
import os
from dotenv import load_dotenv
import requests

# Load environment variables
load_dotenv()

api_key = os.getenv("STEAM_API_KEY")

# Example API request (get player summaries)
steam_id = os.getenv("STEAM_ID")
url = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={api_key}&steamids={steam_id}"

response = requests.get(url)
data = response.json()

print(data)


{'response': {'players': [{'steamid': '76561199690294419', 'communityvisibilitystate': 3, 'profilestate': 1, 'personaname': 'brendan.rb', 'profileurl': 'https://steamcommunity.com/profiles/76561199690294419/', 'avatar': 'https://avatars.steamstatic.com/7ae537b6b7b7b09e68f52e76cdb8f0727f5cb270.jpg', 'avatarmedium': 'https://avatars.steamstatic.com/7ae537b6b7b7b09e68f52e76cdb8f0727f5cb270_medium.jpg', 'avatarfull': 'https://avatars.steamstatic.com/7ae537b6b7b7b09e68f52e76cdb8f0727f5cb270_full.jpg', 'avatarhash': '7ae537b6b7b7b09e68f52e76cdb8f0727f5cb270', 'lastlogoff': 1762918447, 'personastate': 1, 'primaryclanid': '103582791429521408', 'timecreated': 1716259251, 'personastateflags': 0}]}}


In [5]:
import requests

# Get list of all Steam apps
url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
response = requests.get(url)
data = response.json()

# Example: print first 5 games
for app in data['applist']['apps'][:5]:
    print(app['appid'], app['name'])


5 Dedicated Server
7 Steam Client
8 winui2
10 Counter-Strike
20 Team Fortress Classic


In [8]:
len(data['applist']['apps'])

276723

In [4]:
appid = 570  # Example: Dota 2
url = f"https://store.steampowered.com/api/appdetails?appids={appid}"

response = requests.get(url)
data = response.json()

# Access the game info
game_data = data[str(appid)]['data']
print("Name:", game_data['name'])
print("Release date:", game_data['release_date']['date'])
print("Genres:", [g['description'] for g in game_data['genres']])
if 'price_overview' in game_data:
    print("Price:", game_data['price_overview']['final_formatted'])


Name: Dota 2
Release date: Jul 9, 2013
Genres: ['Action', 'Strategy', 'Free To Play']


In [24]:
import time
import pandas as pd
# Helper function to get app details from Steam Store API
def get_game_details(appid):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    response = requests.get(url)
    data = response.json()
    
    if data[str(appid)]['success']:
        game = data[str(appid)]['data']
        genres = [g['description'] for g in game.get('genres', [])]
        release_date = game.get('release_date', {}).get('date', None)
        developers = game.get('developers', [])
        publishers = game.get('publishers', [])
        price = game.get('price_overview', {}).get('final_formatted', None)
        metacritic = game.get('metacritic', {}).get('score', None)
        return {
            "appid": appid,
            "name": game.get('name'),
            "release_date": release_date,
            "genres": genres,
            "developers": developers,
            "publishers": publishers,
            "price": price,
            "metacritic_score": metacritic
        }
    return None

# Step 1: Get a list of all apps
app_list_url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
resp = requests.get(app_list_url)
all_apps = resp.json()['applist']['apps']

# Step 2: For simplicity, pick the first 1000 apps and filter manually
# (You can replace this with top-sellers scraping if desired)
popular_apps = all_apps

games_data = []

for i, app in enumerate(popular_apps):
    try:
        details = get_game_details(app['appid'])
        if details:
            games_data.append(details)
            # print(f"{i+1}: Fetched {details['name']}")
        time.sleep(1)  # rate limiting
        if len(games_data) >= 300:  # stop after 300 games
            break
    except Exception as e:
        print(f"Error fetching appid {app['appid']}: {e}")

# Step 3: Save to CSV
df = pd.DataFrame(games_data)
# df.to_csv("steam_top_300_games.csv", index=False)
# print("Saved top 300 Steam games to steam_top_300_games.csv")


Error fetching appid 5: Expecting value: line 1 column 1 (char 0)
Error fetching appid 7: Expecting value: line 1 column 1 (char 0)
Error fetching appid 8: Expecting value: line 1 column 1 (char 0)
Error fetching appid 10: Expecting value: line 1 column 1 (char 0)
Error fetching appid 20: Expecting value: line 1 column 1 (char 0)
Error fetching appid 30: Expecting value: line 1 column 1 (char 0)
Error fetching appid 40: Expecting value: line 1 column 1 (char 0)
Error fetching appid 50: Expecting value: line 1 column 1 (char 0)
Error fetching appid 60: Expecting value: line 1 column 1 (char 0)
Error fetching appid 70: Expecting value: line 1 column 1 (char 0)
Error fetching appid 80: Expecting value: line 1 column 1 (char 0)
Error fetching appid 90: Expecting value: line 1 column 1 (char 0)
Error fetching appid 92: Expecting value: line 1 column 1 (char 0)
Error fetching appid 215: Expecting value: line 1 column 1 (char 0)
Error fetching appid 218: Expecting value: line 1 column 1 (char

In [23]:
len(all_apps)

276729

In [25]:
len(df)

300

In [26]:
df.to_parquet("steam_top_300_games.parquet", index=False)
print("Saved top 300 Steam games to steam_top_300_games.parquet")

Saved top 300 Steam games to steam_top_300_games.parquet


In [14]:
import os
from googleapiclient.discovery import build
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("YOUTUBE_API_KEY")

youtube = build('youtube', 'v3', developerKey=api_key)

# Example: search for "Dota 2"
request = youtube.search().list(q="Dota 2", part="snippet", type="video", maxResults=1)
response = request.execute()
print(response)


{'kind': 'youtube#searchListResponse', 'etag': 'ZBVWJQWKfZ2DcjeEFMx4vBLjXfQ', 'nextPageToken': 'CAEQAA', 'regionCode': 'US', 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 1}, 'items': [{'kind': 'youtube#searchResult', 'etag': 'qGK00eTSmUwnBatyH4leFqq2tlk', 'id': {'kind': 'youtube#video', 'videoId': 'xhfs_1aMdMQ'}, 'snippet': {'publishedAt': '2025-11-09T14:14:23Z', 'channelId': 'UCAvIC2XmBLLXFPdveirTrmw', 'title': 'EPIC GRAND FINAL! TUNDRA vs FALCONS - Official Highlights - BLAST Slam IV Dota 2', 'description': 'Check out the match page here for in depth stats, team and player info: ...', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/xhfs_1aMdMQ/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/xhfs_1aMdMQ/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/xhfs_1aMdMQ/hqdefault.jpg', 'width': 480, 'height': 360}}, 'channelTitle': 'BLAST Slam Dota 2', 'liveBroadcastContent': 'none', 'publishTime':

In [27]:
import os
import pandas as pd
import time
from dotenv import load_dotenv
from googleapiclient.discovery import build

# Load API key from .env
load_dotenv()
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY")

# Build YouTube API client
youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)

# Load your Steam top 300 games CSV
games_df = pd.read_parquet("steam_top_300_games.parquet")

# Function to fetch YouTube stats for a game
def get_youtube_stats(game_name, max_results=5):
    """
    Returns total views and number of top videos for a given game name
    """
    try:
        search_request = youtube.search().list(
            q=game_name,
            part="snippet",
            type="video",
            maxResults=max_results,
            order="viewCount"
        )
        search_response = search_request.execute()
        video_items = search_response.get("items", [])
        video_count = len(video_items)
        total_views = 0

        for item in video_items:
            video_id = item["id"]["videoId"]
            stats_request = youtube.videos().list(
                part="statistics",
                id=video_id
            )
            stats_response = stats_request.execute()
            view_count = int(stats_response['items'][0]['statistics'].get('viewCount', 0))
            total_views += view_count

        return {"video_count": video_count, "total_views": total_views}

    except Exception as e:
        print(f"Error fetching stats for {game_name}: {e}")
        return {"video_count": 0, "total_views": 0}

# Loop through all games and collect stats
youtube_stats = []
for i, game in enumerate(games_df['name']):
    stats = get_youtube_stats(game)
    youtube_stats.append(stats)
    print(f"{i+1}/{len(games_df)}: Fetched stats for {game}")
    time.sleep(1)  # rate-limit to avoid exceeding quota

# Convert to DataFrame
youtube_df = pd.DataFrame(youtube_stats)

# Merge YouTube stats with Steam dataset
merged_df = pd.concat([games_df, youtube_df], axis=1)

# Save merged dataset
merged_df.to_parquet("steam_top_300_with_youtube.parquet", index=False)
print("✅ Saved merged dataset: steam_top_300_with_youtube.csv")


1/300: Fetched stats for Counter-Strike: Condition Zero
2/300: Fetched stats for Half-Life: Blue Shift
3/300: Fetched stats for Counter-Strike Beta
4/300: Fetched stats for Portal 2
5/300: Fetched stats for Alien Swarm
6/300: Fetched stats for X-Blades
7/300: Fetched stats for Two Worlds II HD
8/300: Fetched stats for Two Worlds II Castle Defense
9/300: Fetched stats for Sid Meier's Railroads!
10/300: Fetched stats for Railroad Tycoon 3
11/300: Fetched stats for Railroad Tycoon II Platinum
12/300: Fetched stats for X-COM: Terror From the Deep
13/300: Fetched stats for X-COM: Apocalypse
14/300: Fetched stats for BioShock™
15/300: Fetched stats for BioShock Demo
16/300: Fetched stats for X-COM: Interceptor
17/300: Fetched stats for NBA 2K9
18/300: Fetched stats for X-COM: UFO Defense
19/300: Fetched stats for X-COM: Enforcer
20/300: Fetched stats for MLB® Front Office Manager
21/300: Fetched stats for Stubbs the Zombie in Rebel Without a Pulse
22/300: Fetched stats for Top Spin 2
23/300:

In [28]:
df1 = pd.read_parquet("steam_top_300_with_youtube.parquet")

In [29]:
df1.head()

,appid,name,release_date,genres,developers,publishers,price,metacritic_score,video_count,total_views
0,100,Counter-Strike: Condition Zero,"Mar 1, 2004",[Action],[Valve],[Valve],$9.99,65.0,5,51092231
1,130,Half-Life: Blue Shift,"1 Jun, 2001",[Action],[Gearbox Software],[Valve],₹ 250,71.0,5,12370177
2,150,Counter-Strike Beta,,[Action],[],[],None,NaN,5,47480546
3,620,Portal 2,"18 Apr, 2011","[Action, Adventure]",[Valve],[Valve],"₩ 11,000",95.0,5,152230849
4,630,Alien Swarm,"Jul 19, 2010","[Action, Free To Play]",[Valve],[Valve],None,77.0,5,122606613


In [30]:
len(df1)

300